In [1]:
import json
import glob
import re
import pandas as pd
import os

In [2]:
root="/home/jovyan/work"
analysis_results_path = root + "/analysis/results/"

In [3]:
real_bics = pd.read_csv('groundOfTruth.csv').to_dict('records')

In [123]:
abs_results = []
percentage_results = []
inner_join = []

# FOR EACH BIC
for bic in real_bics:

    bic_hash = bic['Inducing Change']
    regression_id = "{project}_Bug_{id}".format(project=bic['Project'],id=bic['Bug ID'])
    
    abs_result = {
        'id': regression_id,
        'real_bic': bic_hash,
        'regression': False
    }
    
    percentage_result = {
        'id': regression_id,
        'real_bic': bic_hash,
        'regression': False
    }
    
    # OUR RESULTS (RegressionSeeker)
    our_result = False
    result_path = "{path}/{project}/Bug_{id}/bug_result.json".format(path=analysis_results_path, project=bic['Project'], id=bic['Bug ID'])
    if os.path.isfile(result_path):
        with open(result_path) as f:
            rs_candidates = list(map(lambda c: c[1],json.load(f)['BIC_candidates']))
            
            if len(rs_candidates) == 1: 
                bic_info = bic.copy()
                bic_info[''] = bic['Inducing Change'] == rs_candidates[0]
                inner_join.append(bic_info)
            if len(rs_candidates) > 0: 
                abs_result['regression'] = True
                percentage_result['regression'] = True
            abs_result['RS'] = bic_hash in rs_candidates
            percentage_result['RS'] = 1/len(rs_candidates) if bic_hash in rs_candidates else 0
                        
    # SZZ UNLEASHED
    szz_unleashed_result_path = root+"/results/szz/SZZUnleashed/{id}_SZZUnleashed/results/fix_and_introducers_pairs.json".format(id=regression_id)
    if os.path.isfile(szz_unleashed_result_path):
        with open(szz_unleashed_result_path) as f:
            szz_unleashed_candiates = list(set(map(lambda c: c[1], json.load(f))))
            #print(regression_id, bic_hash in szz_unleashed_candiates, szz_unleashed_candiates)
            abs_result['SZZ_UNLEASHED'] = bic_hash in szz_unleashed_candiates
            percentage_result['SZZ_UNLEASHED'] = 1/len(szz_unleashed_candiates) if bic_hash in szz_unleashed_candiates else 0

    # OPEN SZZ
    openszz_result_path = root+"/results/szz/OpenSZZ/{id}_OpenSZZ/suspects.json".format(id=regression_id)
    if os.path.isfile(openszz_result_path):
        with open(openszz_result_path) as f:
            openszz_candiates = list(set(json.load(f)))
            abs_result['OPEN_SZZ'] = bic_hash in openszz_candiates
            percentage_result['OPEN_SZZ'] = 1/len(openszz_candiates) if bic_hash in openszz_candiates else 0
    
    # PYSZZ
    for alg in ['ag', 'l', 'r', 'ma', 'ra']:
        pyszz_result_path = root+"/results/szz/PySZZ_{alg}/{id}_PySZZ_{alg}/result.json".format(id=regression_id,alg=alg)
        if os.path.isfile(pyszz_result_path):
            with open(pyszz_result_path) as f:
                pyszz_candiates = list(set(json.load(f)[0]['inducing_commit_hash']))
                abs_result['PYSZZ_'+alg] = bic_hash in pyszz_candiates
                percentage_result['PYSZZ_'+alg] = 1/len(pyszz_candiates) if bic_hash in pyszz_candiates else 0
                
    # RS + PYSZZ_{alg}
    for alg in ['ag', 'ma']:
        pyszz_result_path = root+"/results/szz/PySZZ_{alg}/{id}_PySZZ_{alg}/result.json".format(id=regression_id,alg=alg)
        if os.path.isfile(result_path) and os.path.isfile(pyszz_result_path):
            with open(pyszz_result_path) as f:
                pyszz_candiates = list(set(json.load(f)[0]['inducing_commit_hash']))
            with open(result_path) as f:
                rs_candidates = list(map(lambda c: c[1],json.load(f)['BIC_candidates']))

            if len(pyszz_candiates) == 0:
                abs_result['RS-PYSZZ_'+alg] = bic_hash in rs_candidates
                percentage_result['RS-PYSZZ_'+alg] = 1/len(rs_candidates) if bic_hash in rs_candidates else 0
            elif len(pyszz_candiates) == 1:
                abs_result['RS-PYSZZ_'+alg] = bic_hash in pyszz_candiates
                percentage_result['RS-PYSZZ_'+alg] = 1 if bic_hash in pyszz_candiates else 0
            elif len(pyszz_candiates) > 1:
                intersection_candidates = set.intersection(set(pyszz_candiates), set(rs_candidates))
                if len(intersection_candidates) == 0:
                    all_candidates = pyszz_candiates + rs_candidates
                    abs_result['RS-PYSZZ_'+alg] = bic_hash in all_candidates
                    percentage_result['RS-PYSZZ_'+alg] = 1/len(all_candidates) if bic_hash in all_candidates else 0
                else:
                    abs_result['RS-PYSZZ_'+alg] = bic_hash in intersection_candidates
                    percentage_result['RS-PYSZZ_'+alg] = 1/len(intersection_candidates) if bic_hash in intersection_candidates else 0
                
    # ADD RESULTS
    abs_results.append(abs_result)
    percentage_results.append(percentage_result)

abs_results_df = pd.DataFrame(abs_results)
percentage_results_df = pd.DataFrame(percentage_results)

In [124]:
pd.DataFrame(inner_join)

,Project,Bug ID,Inducing Change,
0,Closure,21,dbf6ea95477810188582b9e9ac6c9645717cbe95,True
1,Closure,31,a4c526dae1537f027f429145656ffb849699c1fc,True
2,Closure,61,e9a1cc9c431416354dec3dcb393affd7f41c6596,True
3,Closure,62,22a1fcd79bd2bfbfdc9e80d5162bdf19065f9e5e,True
4,Closure,104,1be1c2636ef14071afa7da5cb8988972bd90149f,True
5,Closure,113,87071ddf324c3885361f0375b84e15c7cd62283d,True
6,Closure,114,f322be0e576d5e2114cb59c0a6537197997b9c59,False
7,Closure,115,b80dfd3b0e5e4b490b7afb469320d18bac4520b4,True
8,Closure,120,000ea0f8d6684bb9ff84a483d4ff44952b29d3f7,True
9,Lang,13,01ab63a0f1d4b8d17d86dc4be56cbc7b92154a48,True


In [107]:
def getSummary(absolute, percentage):
    results_summary = []
    for alg in ['RS', 'RS-PYSZZ_ag', 'RS-PYSZZ_ma', 'SZZ_UNLEASHED', 'OPEN_SZZ', 'PYSZZ_ag', 'PYSZZ_l', 'PYSZZ_r', 'PYSZZ_ma', 'PYSZZ_ra' ]:
        results_summary.append({
            'Algorithm': alg,
            'Absolute': "{:.2f}".format((absolute[alg] == True).sum()/len(absolute.index)),
            'Relative': "{:.2f}".format((percentage[alg] == True).sum()/len(percentage.index))
        })
    results_summary_df = pd.DataFrame(results_summary).set_index('Algorithm')
    return results_summary_df

In [108]:
getSummary(abs_results_df, percentage_results_df)

,Absolute,Relative
Algorithm,,
RS,0.29,0.24
RS-PYSZZ_ag,0.41,0.37
RS-PYSZZ_ma,0.44,0.34
SZZ_UNLEASHED,0.02,0.00
OPEN_SZZ,0.20,0.15
PYSZZ_ag,0.37,0.32
PYSZZ_l,0.17,0.17
PYSZZ_r,0.17,0.17
PYSZZ_ma,0.40,0.22


In [109]:
only_regressions_abs = abs_results_df[abs_results_df['regression']==True]
only_regressions_relative = percentage_results_df[percentage_results_df['regression']==True]
getSummary(only_regressions_abs, only_regressions_relative)

,Absolute,Relative
Algorithm,,
RS,0.92,0.77
RS-PYSZZ_ag,0.77,0.65
RS-PYSZZ_ma,0.81,0.73
SZZ_UNLEASHED,0.00,0.00
OPEN_SZZ,0.23,0.12
PYSZZ_ag,0.62,0.50
PYSZZ_l,0.12,0.12
PYSZZ_r,0.15,0.15
PYSZZ_ma,0.69,0.35
